In [18]:
import pandas as pd
import torch
from tqdm import tqdm
import torch.nn as nn
from torch.utils.data import Dataset
from torch.utils.data import random_split
from transformers import BertTokenizer

In [19]:
df = pd.read_csv('data/AI_Human.csv')
df.head()

,text,generated
0,Cars. Cars have been around since they became ...,0.0
1,Transportation is a large necessity in most co...,0.0
2,"""America's love affair with it's vehicles seem...",0.0
3,How often do you ride in a car? Do you drive a...,0.0
4,Cars are a wonderful thing. They are perhaps o...,0.0


In [20]:
# Assuming df is your original dataframe
df_zero = df[df['generated'] == 0]
df_one = df[df['generated'] == 1]

# Sample 5000 rows from each dataframe
df_zero_sampled = df_zero.sample(5000, random_state=1)
df_one_sampled = df_one.sample(5000, random_state=1)

# Concatenate the two dataframes
df = pd.concat([df_zero_sampled, df_one_sampled])
df.reset_index(inplace=True)

In [21]:
# Determining the max length (in words) of rows of the data
maxlen = df['text'].apply(lambda x: len(x.split())).max()
print(f'Max length: {maxlen}')

Max length: 1642


In [22]:
class TextDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.text = dataframe.text
        self.targets = dataframe.generated
        self.max_len = max_len

    def __len__(self):
        return len(self.text)

    def __getitem__(self, index):
        text = str(self.text[index])
        text = " ".join(text.split())

        inputs = self.tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True,
            return_token_type_ids=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']

        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'targets': torch.tensor(self.targets[index], dtype=torch.float)
        }

In [23]:
# Initialize the tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Create the Dataset
dataset = TextDataset(df, tokenizer, max_len=maxlen)

In [24]:
def create_data_loaders(dataset, train_prop=0.8, val_prop=0.1, test_prop=0.1, batch_size=128):
    train_len = int(train_prop * len(dataset))
    val_len = int(val_prop * len(dataset))
    test_len = int(test_prop * len(dataset))

    train_set, val_set, test_set = random_split(dataset, [train_len, val_len, test_len])

    train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, shuffle=True)
    val_loader = torch.utils.data.DataLoader(val_set, batch_size=batch_size, shuffle=False)
    test_loader = torch.utils.data.DataLoader(test_set, batch_size=batch_size, shuffle=False)

    return train_loader, val_loader, test_loader

In [25]:
train_loader, val_loader, test_loader = create_data_loaders(dataset)

In [26]:
class LSTMClassifier(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, 
                 bidirectional, dropout):
        super().__init__()
        
        # Embedding layer
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        
        # LSTM layer
        self.lstm = nn.LSTM(embedding_dim, 
                           hidden_dim, 
                           num_layers=n_layers, 
                           bidirectional=bidirectional, 
                           dropout=dropout,
                           batch_first=True)
        
        # Dense layer
        self.fc = nn.Linear(hidden_dim * 2, output_dim)
        
        # Activation function
        self.act = nn.Sigmoid()
        
    def forward(self, text, text_lengths):
        
        # text = [batch size, sent_length]
        embedded = self.embedding(text)
        # embedded = [batch size, sent_len, emb dim]
      
        # packed sequence
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths.cpu(), batch_first=True)
        
        packed_output, (hidden, cell) = self.lstm(packed_embedded)

        #hidden = [batch size, num layers * num directions,hid dim]
        #cell = [batch size, num layers * num directions,hid dim]
    
        #concat the final forward and backward hidden state
        hidden = torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1)
                
        #hidden = [batch size, hid dim * num directions]
        dense_outputs=self.fc(hidden)

        #Final activation function
        outputs=self.act(dense_outputs)
        
        return outputs

In [27]:
# Define hyperparameters
vocab_size = len(tokenizer.vocab)
embedding_dim = 100
hidden_dim = 256
output_dim = 1
n_layers = 2
bidirectional = True
dropout = 0.2

# Create the model
model = LSTMClassifier(vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, 
                 bidirectional, dropout)

In [28]:
# Check if CUDA is available and set device to GPU if it is, else CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [29]:
# Define the loss function
criterion = nn.BCELoss()

# Define the optimizer
optimizer = torch.optim.Adam(model.parameters())

# Move the model and loss function to GPU
model = model.to(device)
criterion = criterion.to(device)

In [31]:
# Number of training epochs
epochs = 10

model.train()

# Start the training loop
for epoch in range(epochs):
    for batch in train_loader:
        # Zero the gradients
        optimizer.zero_grad()

        # Forward pass
        predictions = model(batch['ids']).squeeze()
        loss = criterion(predictions, batch['targets'])

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

TypeError: forward() missing 1 required positional argument: 'text_lengths'